In [ ]:
# ! pip install pdfplumber

In [3]:
import re
import pdfplumber
import pandas as pd
from collections import namedtuple

In [38]:
Line = namedtuple('Line', 'company_id company_name doctype reference currency voucher inv_date due_date open_amt_tc open_amt_bc current months1 months2 months3')

In [13]:
company_re = re.compile(r'(V\d+) (.*) Phone:')# to get two elements from the line
line_re = re.compile(r'\d{2}/\d{2}/\d{4} \d{2}/\d{2}/\d{4}')# two digits followed by slash, -\\-, four digits follow by the slash
# looking for the pattern date and date

In [14]:
# Find this line

In [15]:
company_re.search('V50012 Company A LTD Phone:2123232323').group(1)

'V50012'

In [16]:
line_re.search('478 USD BFS030OB/000000057 11/03/2014 12/30/2014 263.52 263.52 0.00 0.00 0.00 263.52')

<re.Match object; span=(27, 48), match='11/03/2014 12/30/2014'>

In [17]:
# To split the whole line

In [18]:
'478 USD BFS030OB/000000057 11/03/2014 12/30/2014 263.52 263.52 0.00 0.00 0.00 263.52'.split()

['478',
 'USD',
 'BFS030OB/000000057',
 '11/03/2014',
 '12/30/2014',
 '263.52',
 '263.52',
 '0.00',
 '0.00',
 '0.00',
 '263.52']

In [50]:
file = 'Sample.pdf'

In [39]:
lines = []
total_check = 0

with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()
        for line in text.split('\n'):
            comp = company_re.search(line)
            if comp:
                vend_no, vend_name = comp.group(1), comp.group(2)
            elif line.startswith('INVOICES'):
                doctype = 'INVOICE'
                
            elif line.startswith('CREDITNOTES'):
                doctype = 'CREDITNOTE'
                
            elif line_re.search(line):
                items = line.split()
                lines.append(Line(vend_no, vend_name, doctype, *items))
                
            elif line.startswith('Supplier total'):
                tot = float(line.split()[2].replace(',', ''))
                total_check += tot

lines[:3]

In [40]:
df = pd.DataFrame(lines)
df.head()

company_id                    company_name     doctype   reference currency  \
0     V50012                   Company A LTD  CREDITNOTE         478      USD   
1     V50012            CARGO SAFETRAVEL LTD     INVOICE  J1062628/1      USD   
2     V50012            CARGO SAFETRAVEL LTD     INVOICE  J1062638/1      USD   
3     V50014            ChaseB REGISTER USDA     INVOICE  110-135633      USD   
4     V50015  METALS CHEMICALS GROUP LIMITED     INVOICE    SI191433      USD   

              voucher    inv_date    due_date open_amt_tc open_amt_bc  \
0  BFS030OB/000000057  11/03/2014  12/30/2014      263.52      263.52   
1    BFS026/000001137  03/30/2019  03/30/2019     -102.00     -102.00   
2    BFS026/000001136  02/28/2019  03/15/2019     -204.00     -204.00   
3    BFS026/000001064  02/05/2019  04/29/2019   -4,665.00   -4,665.00   
4    BFS026/000000908  02/12/2019  03/30/2019     -198.00     -198.00   

     current  months1 months2 months3  
0       0.00     0.00    0.00  263.52  
1       0.00  -102.00    0.00    0.00  
2       0.00  -204.00    0.00    0.00  
3  -4,665.00     0.00    0.00    0.00  
4       0.00  -198.00    0.00    0.00

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   company_id    131 non-null    object        
 1   company_name  131 non-null    object        
 2   doctype       131 non-null    object        
 3   reference     131 non-null    object        
 4   currency      131 non-null    object        
 5   voucher       131 non-null    object        
 6   inv_date      131 non-null    datetime64[ns]
 7   due_date      131 non-null    datetime64[ns]
 8   open_amt_tc   131 non-null    float64       
 9   open_amt_bc   131 non-null    float64       
 10  current       131 non-null    float64       
 11  months1       131 non-null    float64       
 12  months2       131 non-null    float64       
 13  months3       131 non-null    float64       
dtypes: datetime64[ns](2), float64(6), object(6)
memory usage: 14.5+ KB


In [42]:
df['inv_date'] = pd.to_datetime(df['inv_date'])
df['due_date'] = pd.to_datetime(df['inv_date'])

In [ ]:
# convert the amounts in last six col

In [46]:
for col in df.columns[-6:]:
    df[col] = df[col].map(lambda x: float(str(x).replace(',', '')))

In [45]:
df['open_amt_bc'].sum()

-129580.73999999999

In [48]:
total_check

-129580.73999999999

In [49]:
df.to_csv('invoices.csv', index=False)